In [1]:
import pandas as pd
import numpy as np
import gc
import torch
from random import sample

gc.enable()
gc.collect()

0

In [2]:
logs_0708 = pd.read_parquet('./data/logs_df_2024-07-08.parquet')

In [3]:
df_stats = pd.read_parquet('./data/video_stat.parquet')

In [4]:
logs = pd.merge(logs_0708, df_stats[['video_id','v_duration']],on='video_id', how='left')

In [5]:
logs['is_long_watch'] = logs.apply(
    lambda row: 1 if (
        (row['v_duration'] > 300 and row['watchtime'] / row['v_duration'] > 0.25) or
        (row['v_duration'] <= 300 and row['watchtime'] > 30) or
        (row['v_duration'] <= 30 and (row['watchtime'] != 0 and row['v_duration'] != 0 and row['watchtime'] / row['v_duration'] > 0.75))
    ) else -1,
    axis=1
)

In [6]:
watchtime_sum_by_user = logs.groupby('user_id')['watchtime'].sum().reset_index()
watchtime_sum_by_user

,user_id,watchtime
0,0000020c-f22d-428f-a780-14cb34612649,192
1,000002ca-30be-4bfd-8979-23d80fa5b924,31
2,000003e8-2dee-49be-8c41-9082ce7a3af3,817
3,0000062a-9401-4d9d-b2ca-4a54d18c5158,161
4,00000823-4e57-4699-925f-082efd9fc694,20
...,...,...
7561003,fffffa62-02a8-4500-9862-82128d8521b1,3423
7561004,fffffb0b-32bc-4f5c-a910-430c0fe9e5c9,2146
7561005,fffffde2-df64-4b69-bd29-945eb883de0c,82
7561006,fffffded-e124-4080-b929-4c51e3b701d5,4960


In [7]:
watchtime_sum_by_user = watchtime_sum_by_user.set_index('user_id')
watchtime_sum_by_user

,watchtime
user_id,
0000020c-f22d-428f-a780-14cb34612649,192
000002ca-30be-4bfd-8979-23d80fa5b924,31
000003e8-2dee-49be-8c41-9082ce7a3af3,817
0000062a-9401-4d9d-b2ca-4a54d18c5158,161
00000823-4e57-4699-925f-082efd9fc694,20
...,...
fffffa62-02a8-4500-9862-82128d8521b1,3423
fffffb0b-32bc-4f5c-a910-430c0fe9e5c9,2146
fffffde2-df64-4b69-bd29-945eb883de0c,82


In [12]:
users_to_drop = watchtime_sum_by_user.loc[(watchtime_sum_by_user.watchtime > 43200) | (watchtime_sum_by_user.watchtime <=2.5)]
users_to_drop

,watchtime
user_id,
00002552-5a5f-4c55-b71d-2ef6c6da45c1,0
00002f8c-70a8-4eb4-8746-2bac9e3e386b,0
00003a67-03ed-4f2e-b276-5bfc27158af9,0
0000458c-2ff4-49cf-909f-bf4ea5cfff57,2
00004d29-884d-4b73-b4a6-52d6bb0e40a9,1
...,...
ffffc354-4d70-4434-acb4-a6b2183b4577,1
ffffe85b-c767-4baa-91e7-ce3567159f6e,2
ffffed89-cce0-4332-8ba8-fe1b3cb3ea84,0


In [13]:
filtered_df = logs[~logs['user_id'].isin(users_to_drop)]

In [14]:
filtered_df

,event_timestamp,user_id,region,city,video_id,watchtime,v_duration,is_long_watch
0,2024-07-08 16:13:59+03:00,5ab7c3c3-2ba8-4d61-b81e-8722d1249a7c,d0a727cb-ac6b-4bf7-8449-73a5773ee817,daa223d9-5a07-4723-8b1c-e0660fb50036,6ba4b910-2b84-462e-92c7-477d29df3da0,1,45.212,-1
1,2024-07-08 06:29:48+03:00,ad8fdd37-b467-473a-8fd8-121013752abd,1252b9d0-d457-4967-b0a8-c3b3fc064630,d228cf4d-daae-4a4c-b9e0-aa64d8c54352,a766398d-ae80-4e6d-8da6-dc306903f8b0,673,2557.889,1
2,2024-07-08 19:58:10+03:00,dcfcfe10-dfd1-4d27-a72b-9c324d91224e,5e72a4fe-48c1-4b6c-9b85-078f490357dd,c8910e58-e5da-4ba9-bb99-1c8152ef2942,c36d9722-e24c-4cc1-b4e6-692eb134eb91,16,3906.115,-1
3,2024-07-08 10:09:43+03:00,0a169743-1bed-4ce8-80dd-c5b770b8737f,57ca7ea4-0a03-465d-9302-6ec6b350eefd,633d9f2d-7453-417d-8454-0ce9f935b693,ec9925eb-5d7d-4b7a-8646-aa68761adffd,2,75.543,-1
4,2024-07-08 19:15:16+03:00,08ff79ea-afa2-4562-b808-380f291239a1,4496e8c2-fe04-4654-995c-4e39173f0a77,62453693-e6de-4ff7-a06c-1ba765266f35,9d000f21-5760-481c-9b66-552dd28551a6,5749,5769.151,1
...,...,...,...,...,...,...,...,...
17224931,2024-07-08 15:21:52+03:00,d25fbd66-09ed-49d4-b19e-7662918b5670,8a159dca-6240-4954-b39b-58734dc8d8c1,feeacfe9-3207-41b5-bed8-223ec946ef25,c46484ef-72fb-4d63-9e48-2c73d2f449cc,1254,5313.667,-1
17224932,2024-07-08 08:59:33+03:00,6670e439-476f-49bd-b1f7-3b6aca0d088c,7ef5cdcf-c69e-4124-a17b-a4db22143663,c37340ba-8fc7-4999-940b-8e4f3883182d,110a905d-00de-4fbe-b708-0e436fc82e77,21,21.293,1
17224933,2024-07-08 21:30:01+03:00,0b880ff1-f439-4945-aa8f-25939e4bc47c,f5925ad5-7bc3-4c10-b14b-807f0cdd8c4e,9d91a98f-7a35-48eb-987d-49a7fc43aec4,1492b0d6-62fd-4ee2-b79b-37025dad2946,0,1295.296,-1
17224934,2024-07-08 14:31:50+03:00,efb83425-1a64-42af-9e1b-d3dccee91f41,c3286c6b-68dc-4c41-ad08-5583a5e6b23b,3490d142-7ec4-45dd-9293-3bb07213b3bd,8e02799b-9129-4134-a263-7a81bfcb9b38,8990,9355.009,1


In [15]:
logs = filtered_df.iloc[filtered_df.groupby(by=['user_id', 'video_id'])['watchtime'].idxmax()]

In [16]:
logs = logs.drop(columns=[i for i in logs.columns if i not in ['user_id', 'video_id', 'is_long_watch']])

In [17]:
len(logs['user_id'].unique())

7532552

In [18]:
len(logs['video_id'].unique())

624143

In [19]:
user_counts = logs['user_id'].value_counts()

In [20]:
filtered_users = user_counts[(user_counts >= 15) & (user_counts <= 70)]

In [21]:
filtered_users_sorted = filtered_users.sort_values(ascending=False)

In [22]:
len(filtered_users_sorted)

53072

In [23]:
filtered_users_sorted

user_id
75ac4d2f-7d01-4d14-b971-60f5ca97104e    70
9ff10b88-1d38-4834-b0c8-f92a8db51616    70
18aa2e27-f08a-4ee6-93ea-8f0312ff8589    70
2a91d19e-beaa-4968-bea9-c99b17ff1b53    70
930717c0-ffe3-491d-9e16-6d8e1e291af3    70
                                        ..
02bc706d-9b1b-49fe-aeaa-89fc109074d3    15
9b891605-506a-4687-9275-4071c604f847    15
0826bf15-425a-4b3a-b2b4-beeac443eec6    15
6d9b61f2-0903-47bb-b435-992d5c7b1193    15
0b189566-835e-4c96-8985-d21be48637a3    15
Name: count, Length: 53072, dtype: int64

In [24]:
logs_tmp = logs[logs['user_id'].isin(filtered_users_sorted.index.values)]

In [25]:
len(logs_tmp)

1203238

In [26]:
logs_tmp = logs_tmp.iloc[sample(range(0, len(logs_tmp)), 50000)]
matrix = logs_tmp.pivot(index='user_id', columns='video_id', values='is_long_watch').fillna(0)

In [27]:
pip install implicit

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [28]:
from scipy.sparse import csr_matrix, coo_matrix
from implicit.als import AlternatingLeastSquares

In [29]:
matrix.shape

(31661, 29221)

In [30]:
logs_filtered = logs_tmp#[logs_tmp['is_long_watch'] == 1]

In [31]:
user_id_map = {uid: idx for idx, uid in enumerate(logs_filtered['user_id'].unique())}
video_id_map = {vid: idx for idx, vid in enumerate(logs_filtered['video_id'].unique())}

In [32]:
logs_filtered['user_idx'] = logs_filtered['user_id'].map(user_id_map)
logs_filtered['video_idx'] = logs_filtered['video_id'].map(video_id_map)

In [33]:
user_item_matrix = coo_matrix((np.ones(logs_filtered.shape[0]),
                               (logs_filtered['user_idx'], logs_filtered['video_idx'])))

In [34]:
user_item_matrix_csr = csr_matrix(user_item_matrix)

In [35]:
user_item_matrix_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 50000 stored elements and shape (31661, 29221)>

In [36]:
model = AlternatingLeastSquares(factors=128, regularization=0.1, iterations=50)
model.fit(user_item_matrix)

/home/user1/.local/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
/home/user1/.local/lib/python3.10/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0014538764953613281 seconds
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

In [37]:
user_ids = logs_filtered['user_id'].unique()
user_id = user_ids[0]
user_idx = user_id_map[user_id]
recommended = model.recommend(user_idx, user_item_matrix_csr[user_idx], N=10, filter_already_liked_items=False)

print(f"Recommendations for user {user_id}:")
for video_idx, score in zip(recommended[0], recommended[1]):
    video_id = list(video_id_map.keys())[list(video_id_map.values()).index(video_idx)]
    print(f"Video ID: {video_id}, Score: {score}")

Recommendations for user 604d6eb3-9a40-437d-bb38-41448035177d:
Video ID: 8ba23158-7e25-43e7-9293-2c4870521b98, Score: 0.052347250282764435
Video ID: ea3dfc8b-6dd4-4071-b150-ad54ed030faa, Score: 0.0027495636604726315
Video ID: f8cdd1a4-7a10-42bd-a1a3-c7cb3b82dcf5, Score: 0.002490133047103882
Video ID: 8c7a751b-1d95-469c-bfb9-34fe4ead007f, Score: 0.0024827145971357822
Video ID: 9b0b5e73-9f61-4fa0-950c-b3a9834f5d96, Score: 0.0024821264669299126
Video ID: 39a9d5ae-fbc6-4403-ae2e-ccde127db4ec, Score: 0.002376285847276449
Video ID: 0880bb8a-fac6-4016-8d06-cf9013baa4f7, Score: 0.0023643583990633488
Video ID: a8446e21-fb34-44c0-9db3-5b858d8645f3, Score: 0.002359268721193075
Video ID: b107c1fd-a413-47d8-9a8e-c07b521d3079, Score: 0.002359268721193075
Video ID: d51db9be-1e1f-4fde-9e9b-2a904d9414cd, Score: 0.0023530905600637197
